# Solving The Rocket Equation: Optimal Fuel Expulsion for Maximum Altitude
**Author:** Kareem Al Dahshan  
**Date:** 2025-10-20

**Abstract:** This notebook derives the one-dimensional rocket equation for a variable-mass system including gravity and quadratic aerodynamic drag. It numerically investigates how the timing of fuel expulsion (parametrized by \(n\)) affects the rocket's maximum altitude, and presents plots, derivations, and discussion appropriate for a college-level science project.
---


## Table of Contents
1. Physics derivation and nondimensionalization
2. Model setup and parameters
3. Numerical implementation (runnable code)
4. Results: plots and tables
5. Discussion and conclusions


## 1. Physics derivation and nondimensionalization

Start from conservation of momentum for the rocket and the ejected gas in an inertial frame. For instantaneous mass \(m(t)\) and velocity \(v(t)\) (positive upward), and exhaust speed \(v_g\) relative to the rocket (exhaust is emitted backward), external forces gravity and quadratic drag give:
\[
m\,\frac{dv}{dt} + v_g\,\frac{dm}{dt} = -m g - b\,v|v|.
\]

Rearrange to obtain acceleration:
\[
\frac{dv}{dt} = -g - \frac{b}{m} v|v| - v_g\frac{\dot m}{m}.
\]

Choose dimensionless variables:
\[
\tau=\frac{t}{T_0},\qquad u=\frac{v}{v_g},\qquad x=\frac{y}{v_g T_0},\qquad m=\frac{M}{b v_g T_0}.
\]

With \(\gamma = g T_0 / v_g\) and \(m(\tau)=m_0 z(\tau)\) (where \(m_0=M_0/(b v_g T_0)\)), the dimensionless ODE becomes:
\[
\frac{du}{d\tau} = -\gamma - \frac{u|u|}{m(\tau)} - \frac{1}{m(\tau)}\frac{dm}{d\tau},\qquad \frac{dx}{d\tau}=u.
\]

The fuel model used in this study is the dimensionless remaining mass fraction
\[
z(\tau)=1-0.9\,\tau^n,\qquad \tau\in[0,1],
\]
so \(z(1)=0.1\) and the burn finishes at \(\tau=1\).


## 2. Model setup and parameters

Physical parameter values used in this notebook (typical demonstration values):

- \(T_0 = 40\) s (characteristic burn time)
- \(g = 9.81\) m/s² (gravitational acceleration)
- \(v_g = 500\) m/s (exhaust velocity)
- \(M_0 = 20{,}000\) kg (initial total mass)
- \(b = 0.1\) (drag coefficient scale)

Derived dimensionless groups:
- \(m_0 = M_0/(b v_g T_0)\)
- \(\gamma = g T_0 / v_g\)

These parameters are chosen for demonstration; results illustrate qualitative behavior and scaling rather than a specific flight system.


In [ ]:
# 3. Numerical implementation (runnable code)
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Physical constants and parameters
T0 = 40.0       # seconds (burn duration)
g = 9.81        # m/s^2
vg = 500.0      # m/s
M0 = 2e4        # kg
b = 0.1         # drag coefficient

# Dimensionless groups
m0 = M0 / (b * vg * T0)
gamma = g * T0 / vg

print(f"Dimensionless parameters: m0 = {m0:.4f}, gamma = {gamma:.4f}")

In [ ]:
# Fuel fraction and safe derivative (to avoid tau=0 singularity)
def z_fun(tau, n):
    return 1.0 - 0.9 * (tau**n)

def dzdt_fun_safe(tau, n):
    tau_safe = max(tau, 1e-9)
    return -0.9 * n * (tau_safe ** (n - 1))

# Rocket ODE in dimensionless form: state S = [x, u]
def rocket_odes(tau, S, m0, n, gamma=gamma):
    x, u = S
    tau_safe = max(tau, 1e-9)
    if tau <= 1.0:
        z = z_fun(tau_safe, n)
        dz = dzdt_fun_safe(tau_safe, n)
        dxdt = u
        dvdt = -gamma - (1.0 / (m0 * z)) * u * abs(u) - (1.0 / z) * dz
    else:
        z1 = z_fun(1.0, n)
        dxdt = u
        dvdt = -gamma - (1.0 / (m0 * z1)) * u * abs(u)
    # Prevent motion below ground
    if x <= 0 and dxdt < 0 and dvdt < 0:
        dxdt = 0.0
        dvdt = 0.0
    return [dxdt, dvdt]

In [ ]:
# Sweep over burn-shape parameter n and solve ODE
n_values = np.concatenate((np.linspace(0.5, 2.0, 16), np.array([3.0, 5.0, 8.0, 12.0])))
results = []
t_eval = np.linspace(1e-6, 3.0, 2001)  # integrate through burn and coast

for n in n_values:
    sol = solve_ivp(rocket_odes, [t_eval[0], t_eval[-1]], [0.0, 0.0],
                    args=(m0, n), t_eval=t_eval, rtol=1e-8, atol=1e-10)
    tau = sol.t
    x = sol.y[0]
    u = sol.y[1]
    # burnout values
    u1 = float(np.interp(1.0, tau, u))
    mask = tau <= 1.0
    tau_int = np.concatenate(([0.0], tau[mask]))
    u_int = np.concatenate(([0.0], u[mask]))
    x1 = np.trapz(u_int, tau_int)
    m1 = m0 * z_fun(1.0, n)
    denom = gamma * m1
    if denom <= 0:
        delta_x_coast = 0.0
    else:
        delta_x_coast = 0.5 * m1 * np.log(1.0 + (u1**2) / denom)
    x_max = x1 + delta_x_coast
    y_max = vg * T0 * x_max
    results.append({'n': float(n), 'u1': float(u1), 'x1': float(x1), 'm1': float(m1),
                    'x_max': float(x_max), 'y_max_m': float(y_max), 'tau': tau, 'u_trace': u})

In [ ]:
# Plot 1: maximum altitude vs n
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot([r['n'] for r in results], [r['y_max_m'] for r in results], marker='o')
plt.title('Maximum Altitude vs Burn-Shape Parameter n')
plt.xlabel('n (Burn-shape parameter)')
plt.ylabel('Maximum altitude y_max (m)')
plt.grid(True)
plt.tight_layout()
plt.show()

# Find best n and print
best = max(results, key=lambda r: r['y_max_m'])
print(f"Best n ~ {best['n']:.4f}, y_max = {best['y_max_m']:.1f} m")

# Plot 2-4: velocity profiles for representative n values
selected_ns = [0.7, 1.0, 5.0]
for sel in selected_ns:
    res = min(results, key=lambda r: abs(r['n']-sel))
    plt.figure(figsize=(8,5))
    plt.plot(res['tau'], res['u_trace'], label=f'n={res["n"]:.2f}')
    plt.axvline(1.0, linestyle='--', color='black', linewidth=0.8, label='End of burn')
    plt.xlabel('τ (dimensionless time)')
    plt.ylabel('u = v / v_g (dimensionless velocity)')
    plt.title(f'u(τ) for n ≈ {res["n"]:.2f}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## 4. Results: figures and explanations

**Figure 1 — Maximum altitude vs burn-shape parameter n**  
This figure plots the maximum apex altitude (in meters) computed for each value of n. Each point uses the integrated velocity during burn plus an analytic coasting contribution. The plot reveals an optimal n that maximizes altitude under the chosen drag and gravity parameters.

**Figures 2–4 — Velocity profiles u(τ) for representative n values**  
These figures show the dimensionless velocity u(τ)=v/v_g throughout burn and coast for selected n values:
- **n ≈ 0.7 (front-loaded):** thrust early, sustained upward velocity during the burn, generally favorable under drag.
- **n ≈ 1.0 (uniform):** balanced burn; intermediate performance.
- **n ≈ 5.0 (back-loaded):** thrust concentrated late; the rocket remains slow during most of the burn and achieves lower apex for these parameters.


## 5. Discussion and conclusions

- Gravity losses during a fixed burn time are the same in integrated magnitude; the optimization is driven mainly by aerodynamic drag losses and how they interact with the timing of high velocity.
- Front-loaded burns tend to minimize total drag losses for the parameter set used here because the rocket gains velocity earlier and spends less time at high speed while in dense atmosphere.
- The model can be extended by varying drag coefficient, exhaust velocity, burn time, and including altitude-dependent atmosphere for additional realism.

---
**Prepared by:** Kareem Al Dahshan
